In [8]:
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
import seaborn as sns
import sys
import sys
import re
import os.path

## data import

In [9]:
file_name =  "clean_data_2022-01-30.csv"
folder = "../clean_equity_data/"
data_tmp = pd.read_csv(folder + file_name)

In [20]:
# FUNCTIONS FOR VIEWING SINGLE COMPANY RANKINGS 

def company(company):
    for i in data['Company']:
        if company in i or company in i.lower():
            return data.loc[data['Company'] == i]
    print('No company found')
    
def Company(company):
    for i in data['Company']:
        if company in i or company in i.lower():
            stock = data.loc[data['Company'] == i]
            stock = stock[compact]
            return stock
    print('No company found')

In [10]:
### MOMENTUM
data_mom = data_tmp.copy()
#data_mom = data_mom.loc[(data_mom['Industry'] != 'Holding Companies')]


### GESTALT
data_gestalt = data_tmp.copy()
data_gestalt = data_gestalt.loc[(data_tmp['Sector'] != 'Financials')]

### Momentum Rank

In [11]:
# RANK ON DIFFERENT METRICS 

data_mom['3m Rank'] = data_mom['Return 3m'].rank(ascending = False)
data_mom['6m Rank'] = data_mom['Return 6m'].rank(ascending = False)
data_mom['1y Rank'] = data_mom['Return 1y'].rank(ascending = False)
data_mom['EA Rank'] = data_mom['EA ret'].rank(ascending = False)
data_mom['EA Std Rank'] = data_mom['EA ret std'].rank(ascending = False)
data_mom['Volatility Rank'] = data_mom['1 Year Volatility'].rank(ascending = True)
data_mom['MAD Rank'] = data_mom['1 Year Volatility'].rank(ascending = True)

# MOMENTUM WITH WOLATILITY AND EA
data_mom['Momentum Rank'] = (data_mom['3m Rank'] + data_mom['6m Rank'] +  data_mom['1y Rank'] + data_mom['Volatility Rank'] + data_mom['EA Std Rank']).rank(ascending = True)



### Gestalt Imputation and Rank

In [12]:
# MANAGE NaNs and NEGATIVE EARNINGS  

# set nan to median, only fundamental data
columns = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'ROC', 'ROE',
            'GPA', 'Assets Turn', 'FCFROE', 'Asset Growth']
for i in columns: 
    data_gestalt.loc[data_gestalt[i].isna() ,i] = data_gestalt[i].median()


    
# set negative values to max
for i in ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B']:
    data_gestalt.loc[data_gestalt[i] < 0 ,i] = data_gestalt[i].max()
    
# set nan yield & vol to 0
data_gestalt.loc[data_gestalt['Yield'].isna(),'Yield'] = 0

In [13]:
data_gestalt['3m Rank'] = data_gestalt['Return 3m'].rank(ascending = False)
data_gestalt['6m Rank'] = data_gestalt['Return 6m'].rank(ascending = False)
data_gestalt['1y Rank'] = data_gestalt['Return 1y'].rank(ascending = False)
data_gestalt['EA Rank'] = data_gestalt['EA ret'].rank(ascending = False)
data_gestalt['EA Std Rank'] = data_gestalt['EA ret std'].rank(ascending = False)


data_gestalt['Momentum Rank'] = (data_gestalt['3m Rank'] + data_gestalt['6m Rank'] + 
                                 data_gestalt['1y Rank'] + data_gestalt['EA Std Rank']).rank(ascending = True)



# Ranking where lower value is better
for i in ['P/E', 'P/B', 'P/S', 'P/FCF', 'EV/EBIT', 'Asset Growth']:
    data_gestalt[i +' Rank'] = data_gestalt[i].rank()
    

for i in ['Yield', 'ROE', 'ROC', 'FCFROE', 'GPA', 'Assets Turn']:
    data_gestalt[i + ' Rank'] = (-data_gestalt[i]).rank()
    
# Composite ranks
data_gestalt['Quality Rank'] = (data_gestalt['ROE Rank'] + data_gestalt['ROC Rank'] + data_gestalt['FCFROE Rank'] + 
                        data_gestalt['GPA Rank'] + data_gestalt['Assets Turn Rank']).rank()

data_gestalt['Value Rank'] = (data_gestalt['P/E Rank'] + data_gestalt['P/B Rank'] + data_gestalt['P/S Rank'] + data_gestalt['P/FCF Rank'] + 
                      data_gestalt['EV/EBIT Rank'] + data_gestalt['Asset Growth Rank']).rank()
 

data_gestalt['Gestalt Rank'] = ((data_gestalt['Value Rank'] + data_gestalt['Momentum Rank'] + data_gestalt['Quality Rank'])).rank()



### Data Error Check

In [14]:
#data_mom.isna().sum()
data_gestalt.isna().sum()

Unnamed: 0          0
Börsdata ID         0
Company             0
Industry            0
Volume              0
                   ..
GPA Rank            0
Assets Turn Rank    0
Quality Rank        0
Value Rank          0
Gestalt Rank        0
Length: 61, dtype: int64

### Sorting

In [15]:

#### MOMENTUM DATA FRAME
momentum_tmp = data_mom.sort_values(by=['Momentum Rank'])
#### Gestalt Data Frame
gestalt_tmp = data_gestalt.sort_values(by=['Gestalt Rank'])

In [17]:

compact = ['Company', 'List','Tick', 'Gestalt Rank', 'Quality Rank', 'Value Rank', 'Momentum Rank', 'EA ret std', '1 Year Volatility']
compact_mom = ['Company', 'List','Tick','Momentum Rank','EA ret std', '1 Year Volatility']
# numbers of stocks to select for diff strategies

stocks = 20
stocks_mom = 25
## MOMENTUM ##
momentum = momentum_tmp[0:stocks_mom]
Momentum = momentum[compact_mom]


## TRIPLE SORT ##
gestalt = gestalt_tmp[0:stocks]
gestalt = gestalt[compact]

### Print Stock Selection

In [18]:
#gestalt 
th_props = [
  ('font-size', '14px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]
# Set CSS properties for td elements in dataframe
td_props = [('font-size', '12px'), ('text-align', 'center')]
#set caption props
caption_props = [('color', 'black'),('font-size', '22px'),
        ("text-align", "center"),
        ('font-weight', 'bold')]
# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props)
]
gestalt.index = range(1, len(gestalt)+1)
gestalt_view = (gestalt.style.apply(lambda x: ['background: lightgreen' if x.name in range(1, 10+1) 
                              else '' for i in x], axis=1).set_table_styles(styles)
                 .format({'EA ret std': "{:.1f}",'1 Year Volatility': "{:.1%}"})
                 .set_caption("Gestalt"))
gestalt_view

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
1,Björn Borg,Small Cap,BORG,1.000000,26.000000,53.000000,8.500000,3.2,40.0%
2,BE Group,Small Cap,BEGR,2.000000,42.000000,40.500000,7.000000,2.3,49.9%
3,Clas Ohlson,Mid Cap,CLAS B,3.000000,22.000000,59.000000,15.000000,3.1,34.8%
4,Electra Gruppen,Small Cap,ELEC,4.000000,41.000000,3.000000,53.000000,-1.1,40.8%
5,Nilörngruppen,Small Cap,NIL B,5.500000,13.000000,67.000000,29.000000,4.2,42.7%
6,B3 Consulting,Small Cap,B3,5.500000,27.000000,72.000000,10.000000,2.9,44.2%
7,Transtema,Small Cap,TRANS,7.000000,4.000000,88.000000,20.000000,2.8,50.6%
8,Poolia,Small Cap,POOL B,8.000000,25.000000,84.000000,12.000000,1.3,49.3%
9,Kabe,Small Cap,KABE B,9.000000,73.000000,36.000000,17.000000,1.9,38.2%
10,Nobina,Mid Cap,NOBINA,10.000000,20.000000,74.000000,38.000000,-0.5,35.2%


In [19]:
th_props = [
  ('font-size', '14px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]
# Set CSS properties for td elements in dataframe
td_props = [('font-size', '12px'), ('text-align', 'center')]
#set caption props
caption_props = [('color', 'black'),('font-size', '22px'),
        ("text-align", "center"),
        ('font-weight', 'bold')]
# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props)
]
Momentum.index = range(1,len(Momentum)+1)
Momentum_view = (Momentum.style.apply(lambda x: ['background: lightgreen' if x.name in range(1,10+1) 
                              else '' for i in x], axis=1).set_table_styles(styles).format({'EA ret std': "{:.1f}",'1 Year Volatility': "{:.1%}"})
                 .set_caption("Focused Momentum"))
Momentum_view

,Company,List,Tick,Momentum Rank,EA ret std,1 Year Volatility
1,ALM Equity,First North,ALM,1.000000,3.6,32.6%
2,Clas Ohlson,Mid Cap,CLAS B,2.000000,3.1,34.8%
3,Björn Borg,Small Cap,BORG,3.000000,3.2,40.0%
4,Amasten,First North,AMAST,4.000000,0.6,25.6%
5,Awardit,First North,AWRD,5.000000,2.2,46.4%
6,Raketech,First North,RAKE,6.000000,5.8,45.0%
7,Alcadon Group,First North,ALCA,7.000000,1.3,41.9%
8,Ericsson B,Large Cap,ERIC B,8.000000,3.5,25.7%
9,Hanza Holding,Small Cap,HANZA,9.000000,4.0,58.7%
10,B3 Consulting,Small Cap,B3,10.000000,2.9,44.2%


In [28]:
momentum_tmp[momentum_tmp['Company'] == 'Transtema']

,Unnamed: 0,Börsdata ID,Company,Industry,Volume,P/FCF,Assets Turn,Gross profit,Tot. Assets,ROC,...,1 Year MAD,EA Volatility,3m Rank,6m Rank,1y Rank,EA Rank,EA Std Rank,Volatility Rank,MAD Rank,Momentum Rank
432,432,775,Transtema,Telecom Services,9.62,5.075,2.193,1069.857,746.278,0.59647,...,0.45005,0.361585,214.0,18.0,17.0,41.0,35.0,312.0,312.0,42.0


In [12]:
#### NEGATIVE RANKS

stocks = 20
## MOMENTUM ##
momentum = momentum_tmp[0:stocks]
Momentum = momentum[compact_mom]


## TRIPLE SORT ##
gestalt = gestalt_tmp[0:stocks]
gestalt = gestalt[compact]

neg_momentum = momentum_tmp.tail(stocks)[::-1][compact_mom]
neg_gestalt = gestalt_tmp.tail(stocks)[::-1][compact]

In [13]:
neg_momentum

,Company,List,Tick,Momentum Rank,EA ret std,1 Year Volatility
120,Desenio,First North,DSNO,427.0,-3.629469,0.931052
240,Kancera,First North,KAN,426.0,-4.734334,0.760562
313,Oncopeptides,Mid Cap,ONCO,425.0,-0.731550,2.006452
104,Climeon,First North,CLIME B,424.0,-1.587466,0.798432
142,Enad Global 7,First North,EG7,423.0,-1.272416,0.739580
347,Readly,Mid Cap,READ,422.0,-2.927363,0.539057
434,Vimian Group,First North,VIMIAN,421.0,-2.185936,0.487458
46,Azelio,First North,AZELIO,420.0,-0.782825,0.772498
385,SpectraCure,First North,SPEC,419.0,-0.735108,0.998207
121,Diamyd Medical,First North,DMYD B,418.0,-0.778326,0.617148


In [14]:
neg_gestalt

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
340,Q-Linea,Mid Cap,QLINEA,371.0,365.0,348.0,358.0,-2.510544,0.427852
191,Hansa Biopharma,Mid Cap,HNSA,370.0,366.0,331.0,351.0,-0.862308,0.527858
117,Crunchfish,First North,CFISH,369.0,346.0,363.5,335.0,-2.849253,0.960145
240,Kancera,First North,KAN,368.0,360.0,315.0,369.0,-4.734334,0.760562
113,Cortus Energy,First North,CE,367.0,371.0,365.0,298.0,-0.869394,0.817887
357,Saniona,Small Cap,SANION,366.0,369.0,287.5,357.0,-0.784732,0.551978
97,Cell Impact,First North,CI B,365.0,336.0,360.0,301.5,-1.480502,0.767370
211,Immunovia,Mid Cap,IMMNOV,364.0,362.0,332.5,297.0,-0.358773,0.694951
104,Climeon,First North,CLIME B,363.0,355.0,267.0,365.5,-1.587466,0.798432
362,Scandinavian Enviro,First North,SES,362.0,361.0,353.0,273.0,-0.084326,0.526077


In [15]:
save = "YES"
if save == "YES":
    data_date  = re.search(r'\d{4}-\d{2}-\d{2}', file_name)
    folder = '../clean_equity_data'
    exp_file_name = "rank_data_{}".format(data_date.group(0))+ ".csv"
    data_tmp.to_csv(os.path.join(folder, exp_file_name))